<a href="https://www.kaggle.com/code/wahyudewo/spacex-falcon-9-landing-projection?scriptVersionId=121452813" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
import requests
import pandas as pd
import numpy as np

In [2]:
urls = {
    'launch': 'https://api.spacexdata.com/v4/launches/past',
    'static_launch': 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/API_call_spacex_api.json',
    'capsules': 'https://api.spacexdata.com/v4/capsules',
    'cores': 'https://api.spacexdata.com/v4/cores',
    'rockets' : 'https://api.spacexdata.com/v4/rockets',
    'launchpads' : 'https://api.spacexdata.com/v4/launchpads',
    'payloads' : 'https://api.spacexdata.com/v4/payloads',
}

for url_names, url in urls.items():
    response = requests.get(url).json()
    df_name = f'{url_names}'
    globals()[df_name] = pd.json_normalize(response)

In [3]:
static_launch = static_launch[static_launch.payloads.map(len)==1]
static_launch = static_launch[static_launch.cores.map(len)==1]

static_launch.payloads = static_launch.payloads.map(lambda x : x[0])
static_launch.cores = static_launch.cores.map(lambda x : x[0])

In [4]:
core = pd.json_normalize(static_launch.cores)
core['Outcome'] = core.landing_success.astype('str') + ' ' + core.landing_type.astype('str')

In [5]:
merge = static_launch\
            .merge(rockets.name,
                   how='left',
                   left_on='rocket',
                   right_on=rockets.id,
                   suffixes=('','_rocket'),
                  )\
            .merge(launchpads[['longitude','latitude','name']],
                   how='left',
                   left_on='launchpad',
                   right_on=launchpads.id,
                   suffixes=('','_launchpad'),
                  )\
            .merge(payloads[['mass_kg','orbit']],
                   how='left',
                   left_on='payloads',
                   right_on=payloads.id,
                   suffixes=('','_payloads'),
                  )\
            .merge(core,
                   how='left',
                   left_index=True,
                   right_index=True,
                  )\
            .merge(cores[['block','reuse_count','serial']],
                   how='left',
                   left_on='core',
                   right_on=cores.id,
                  )\

In [6]:
data = merge [[
#     'static_fire_date_utc','static_fire_date_unix', 'tbd', 'net', 'window',
#     'rocket', 'success', 'details', 'crew', 'ships', 'capsules', 'payloads',
#     'launchpad', 'auto_update', 'failures',
    'flight_number',
#     'name',
    'date_utc',
#     'date_unix', 'date_local', 'date_precision', 'upcoming','cores', 'id',
#     'fairings.reused', 'fairings.recovery_attempt','fairings.recovered',
#     'fairings.ships', 'links.patch.small', 'links.patch.large',
#     'links.reddit.campaign', 'links.reddit.launch', 'links.reddit.media',
#     'links.reddit.recovery', 'links.flickr.small', 'links.flickr.original',
#     'links.presskit', 'links.webcast', 'links.youtube_id', 'links.article',
#     'links.wikipedia', 'fairings',
    'name_rocket', 'mass_kg', 'orbit', 'name_launchpad','Outcome',
#     'core',
    'flight', 'gridfins', 'legs', 'reused',
#     'landing_attempt', 'landing_success', 'landing_type',
    'landpad','block', 'reuse_count', 'serial', 'longitude', 'latitude'
]].rename(columns={'flight_number':'FlightNumber',
                   'date_utc':'Date',
                   'name_rocket':'BoosterVersion',
                   'mass_kg':'PayloadMass',
                   'orbit':'Orbit',
                   'name_launchpad':'LaunchSite',
                   'flight':'Flights',
                   'gridfins':'GridFins',
                   'reused':'Reused',
                   'legs':'Legs',
                   'landpad':'LandingPad',
                   'block':'Block',
                   'reuse_count':'ReusedCount',
                   'serial':'Serial',
                   'longitude':'Longitude',
                   'latitude':'Latitude',
                  }
         )
data.Date = pd.to_datetime(data.Date).dt.date.astype('datetime64')

In [7]:
data_falcon9 = data.query('BoosterVersion == "Falcon 9"').reset_index(drop=True)
data_falcon9.loc[:,'FlightNumber'] = list(range(1, data_falcon9.shape[0]+1))

### Data Wrangling

#### Dealing with Missing Value

In [8]:
data_falcon9.isna().sum()

FlightNumber       0
Date               0
BoosterVersion     0
PayloadMass        6
Orbit              0
LaunchSite         0
Outcome            0
Flights            0
GridFins           0
Legs               0
Reused             0
LandingPad        26
Block              0
ReusedCount        0
Serial             0
Longitude          0
Latitude           0
dtype: int64

In [9]:
PayloadMass_mean = data_falcon9.PayloadMass.mean()
data_falcon9.PayloadMass = data_falcon9.PayloadMass.replace(np.nan,PayloadMass_mean)

In [10]:
data_falcon9.isna().sum()

FlightNumber       0
Date               0
BoosterVersion     0
PayloadMass        0
Orbit              0
LaunchSite         0
Outcome            0
Flights            0
GridFins           0
Legs               0
Reused             0
LandingPad        26
Block              0
ReusedCount        0
Serial             0
Longitude          0
Latitude           0
dtype: int64

In [11]:
data_falcon9.to_csv('dataset_part_1.csv', index=False)